In [11]:
from transformers import RobertaModel, AutoTokenizer
from model import RelModel

In [2]:
import torch

In [12]:
tokenizer = AutoTokenizer.from_pretrained("roberta-large")

In [3]:
encoder = RobertaModel.from_pretrained("roberta-large")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
model = RelModel(encoder).to("cuda")

In [129]:
state_dict = torch.load("/home/ubuntu/nlm/nima/Data/Models/boolq_for_rel/BioRel-short/exp2/epoch_17")

In [130]:
model.load_state_dict(state_dict)

<All keys matched successfully>

In [131]:
model = model.eval()

In [132]:
import json
with open("/home/ubuntu/nlm/nima/Data/BioRel/dev.json") as file:
    data = json.load(file)

In [133]:
i = 52321
input_ = tokenizer(" ".join(data[i]['relation'].split("_")), data[i]['sentence'], return_tensors='pt')

In [134]:
dev_data = torch.load('/home/ubuntu/nlm/nima/Data/BioRel/IO-pre-processed-huggingface/type3/input_ids_dev.pt')

In [135]:
torch.where(dev_data[0] == 1)[0][0]

tensor(43)

In [136]:
input_ids

NameError: name 'input_ids' is not defined

In [ ]:
for i, d in enumerate(dev_data):
    m = torch.where(d == 1)[0][0]
    input_ = {
        "input_ids": d[:m].unsqueeze(0).to('cuda'), 
        "attention_mask": torch.ones([1,m]).to('cuda')
    }
        
    head_logit, tail_logit = model(input_)
    head_mask = head_logit.argmax(axis = -1).squeeze()
    tail_mask = tail_logit.argmax(axis = -1).squeeze()
    print(head_mask.sum(), tail_mask.sum())

In [137]:
input_

{'input_ids': tensor([[    0,   417,  1496,  3175,    34,  3059, 41115, 30344,  1082,     2,
             2,   627,  2225,  2656,    19,     5,  3169,     8,   251,    12,
          1279,   775,     9,  1416,     9, 30011,  2182,     8,   112,     6,
         21190,  2943,  1484,    19, 10665,  1668,  7187,    15,    23,     5,
          1494,     9, 10665,     8, 18422,  1988,  6204, 23991,     9,     5,
         14619,    11,  7571,    12, 43768,   479,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [138]:
tokenizer(data[0]['sentence'], return_tensors = 'pt')

{'input_ids': tensor([[    0,  6403, 38315, 26387,   222,    45,  3625,  3327,  2474,    12,
          9981,   405,  2408,  1022,  2156,    53,  1130,    36,   181,   540,
            87,   479,  2546,  4839,  1230,  2408,  3077,    23,    70,    86,
          5788,    11,   258,  7341,   479,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [139]:
i = 64112
input_ = tokenizer(data[i]['relation'], data[i]['sentence'], return_tensors='pt').to('cuda')

In [140]:
head_logit, tail_logit = model(input_)
head_mask = head_logit.argmax(axis = -1).squeeze()
tail_mask = tail_logit.argmax(axis = -1).squeeze()

In [141]:
tokenizer.decode(input_.input_ids[0][head_mask.bool()])

' thrombocytopenia'

In [142]:
tokenizer.decode(input_.input_ids[0][tail_mask.bool()])

' heparin'

In [128]:
data[i]

{'head': {'CUI': 'C0040034',
  'word': 'thrombocytopenia',
  'start': 107,
  'length': 16,
  'split_start': 17},
 'tail': {'CUI': 'C0019134',
  'word': 'heparin',
  'start': 233,
  'length': 7,
  'split_start': 36},
 'sentence': 'careful clinical review suggested that heparin was either the most likely cause of a contributing cause of thrombocytopenia in 5/166 patients ( 3.0 % ) receiving therapeutic heparin and none of the patients who received prophylactic heparin .',
 'relation': 'has_contraindicated_drug',
 'lexical_feature0': 'of|thrombocytopenia|in|5/166|patients|(|3.0|%|)|receiving|therapeutic|heparin|and|none|of|the|patients|who|received|prophylactic|heparin|.',
 'lexical_feature1': 'cause|of|thrombocytopenia|in|5/166|patients|(|3.0|%|)|receiving|therapeutic|heparin|and|none|of|the|patients|who|received|prophylactic|heparin|.|PAD',
 'lexical_feature2': 'contributing|cause|of|thrombocytopenia|in|5/166|patients|(|3.0|%|)|receiving|therapeutic|heparin|and|none|of|the|patients|who|

In [58]:
tokenizer.decode(input_['input_ids'][0])

'<s>disease has associated anatomic site</s></s>the paper deals with the immediate and long-term results of treatment of 173 female and 1,330 male patients with lung cancer operated on at the department of lung and mediastinal tumors of the institute in 1960-1979.</s>'

In [67]:
input_

{'input_ids': tensor([    0,   354,  4760,    30, 10596,  1152,     2,     2,  6403, 38315,
         26387,   222,    45,  3625,  3327,  2474,    12,  9981,   405,  2408,
          1022,  2156,    53,  1130,    36,   181,   540,    87,   479,  2546,
          4839,  1230,  2408,  3077,    23,    70,    86,  5788,    11,   258,
          7341,   479,     2]),
 'attention_mask': tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 1., 1., 1.])}